In [ ]:
pip install comet_ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 42.5 MB/s eta 0:00:00
  Attempting uninstall: python-box
    Found existing installation: python-box 7.3.2
    Uninstalling python-box-7.3.2:
      Successfully uninstalled python-box-7.3.2


In [ ]:
from comet_ml import Experiment
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
#  Comet ML setup
experiment = Experiment(
    api_key="fL21dFQkgKqCFEaarp7WdUpY0",  # Replace with your Comet API key
    project_name="fashion-mnist-demo",
    workspace="emexy"
)

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : remarkable_pitch_2845
COMET INFO:     url                   : https://www.comet.com/emexy/emy1/2a035ec229fd4b888309c1f165b55278
COMET INFO:   Metrics:
COMET INFO:     loss : 0.19257575273513794
COMET INFO:   Others:
COMET INFO:     notebook_url : https://colab.research.google.com/notebook#fileId=1gq8CJPsZamcthSlDlxJCt4kEWYjIiYQ0
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 50
COMET INFO:     learning_rate : 0.5
COMET INFO:     steps         : 100000
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     model graph         : 1
COMET IN

In [ ]:
# Hyperparameters
hyper_params = {
    "learning_rate": 0.001,
    "batch_size": 64,
    "epochs": 5
}
experiment.log_parameters(hyper_params)

In [ ]:
# Data loading & transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=hyper_params["batch_size"], shuffle=True)


In [ ]:
#Model definition
class FashionClassifier(nn.Module):
    def __init__(self):
        super(FashionClassifier, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Training loop
def train(model, loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(loader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")
        experiment.log_metric("loss", avg_loss, step=epoch+1)

In [ ]:
# 🚀 Run training
model = FashionClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=hyper_params["learning_rate"])
train(model, train_loader, criterion, optimizer, hyper_params["epochs"])

Epoch [1/5], Loss: 0.4964
Epoch [2/5], Loss: 0.3810
Epoch [3/5], Loss: 0.3432
Epoch [4/5], Loss: 0.3194
Epoch [5/5], Loss: 0.3002


In [ ]:
# 💾 Save & log model
torch.save(model.state_dict(), "fashion_model.pt")
experiment.log_model("FashionClassifier", "fashion_model.pt")

{'web': 'https://www.comet.com/api/asset/download?assetId=3b1391f762a749e3ae00a480591f9b5e&experimentKey=5ca978c19467481187ea32cc1ba0a11a',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=3b1391f762a749e3ae00a480591f9b5e&experimentKey=5ca978c19467481187ea32cc1ba0a11a',
 'assetId': '3b1391f762a749e3ae00a480591f9b5e'}